#### Imports

In [5]:
# import of our functionality
#from src.symmetry import get_symmetry

# import of external libs
import pandas as pd
import numpy as np  
import os
import matplotlib.pyplot as plt

from os.path import exists

from pathlib import Path

from PIL import Image


In [ ]:
#images = [image for image in os.listdir('../data/images/')]

file_data = '..' + os.sep + 'data' + os.sep +'metadata.csv'
path_image = '..' + os.sep + 'data' + os.sep + 'images' #we have to change it ---> delete raw folder

#Where we will store the features
file_features = 'features/features.csv'

#Read meta-data into a Pandas dataframe
metadata = pd.read_csv(file_data)

images_id = list(metadata['img_id'])



#Read meta-data into a Pandas dataframe
#metadata = pd.read_csv(file_data)

# Extract image IDs and labels from the data. 
#image_id = list(metadata['img_id'])
#label = np.array(metadata['diagnostic'])


In [1]:
############ SYMMETRY ###############

def crop_leison(mask):
    #Takes the mask image and returns the only the leison 

    mask_array = np.where(np.array(mask)==255, 1, 0) #2d array with 0 on black an 1 on white
    
    row_index =  np.where(np.sum(mask_array, axis=1)>0)[0] #all the rows with at least one white element 
    first_row , last_row = row_index[0] , row_index[-1]  #first and last row 
    if (last_row - first_row) %2 != 0:
        last_row += 1 #one extra row to make it even and able to halve it

    col_index =  np.where(np.sum(mask_array, axis=0)>0)[0] #all the col with at least one white element 
    first_col , last_col = col_index[0] , col_index[-1]  #first and last col 
    if (last_col - first_col) %2 != 0:
        last_col += 1

    #center of the leison 
    #xy_center = ((first_col + last_col)/2 , (first_row + last_row)/2) 

    cropped_mask = mask.crop((first_col,first_row,last_col,last_row))

    return(cropped_mask)

def make_bw(mask):
    '''Turns a mask into only black(0) or white(255) pixel values'''
        #why our masks aren't totally bw after labelstudio????

    bw= np.asarray(mask).copy()
    bw[bw<128] = 0
    bw[bw >= 128] = 255

    return Image.fromarray(bw)

def test_assymetry(c_mask):
    #my cropped mask is already even

    width,height = c_mask.size
    #cut in half and fold
    left = c_mask.crop((0, 0, int(width/2), height)) #left part of picture (left, top, right, bottom)
    right = c_mask.crop((int(width/2), 0, width, height)) #right part of picture
    right = right.transpose(Image.FLIP_LEFT_RIGHT) #flip right part to compare

    #difference in both sides
    asym = np.sum(np.where(np.array(left) != np.array(right), 1, 0))
    total_white = (np.sum(np.array(left) + np.array(right)))/255 
   
    return round((asym/total_white) , 3) #the percentage of asymmetry 
        #is it correct to do different/white instead of /total = ((width*height)/2)



def get_asymmetry(mask):
    '''Returns the asymmetry for a given mask by folding it in half from multiple angles'''

    #Need to expand the image to avoid missing part of the leison when rotating the image !!!!
    old_width , old_height = mask.size 
    expanded_im = ImageOps.expand(mask, border = int(old_width/2))

    assym = [test_assymetry(expanded_im.rotate(angle)) for angle in [0,45,90]]   #those angles??

    return round(np.mean(assym), 3)  #mean or min???




In [ ]:
def remove_alpha_channel(image):
    if image.shape[-1] == 4:  # check if there is a 4th channel (alpha)
        image = image[..., :3]  # remove the 4th channel
    return image

In [ ]:
# average color
def get_average_color(masked_image): # image needs to be masked
    r, g, b = masked_image.split() # converting to separate channels  
    r= np.array(r)
    g= np.array(g)
    b= np.array(b) 
    return [round(np.mean(r[r > 0])),round(np.mean(g[g > 0])),round(np.mean(b[b > 0]))]


# color variability
def get_color_variability(masked_image, measure='variance'): # image needs to be masked
    r, g, b = masked_image.split() # converting to separate channels  
    r= np.array(r) 
    g= np.array(g)
    b= np.array(b) 
    if measure == 'variance': rgb=(np.var(r[r > 0]),np.var(g[g > 0]),np.var(b[b > 0]))
    elif measure == 'standard_deviation': rgb=(np.std(r[r > 0]),np.std(g[g > 0]),np.std(b[b > 0]))
    else: return 
    return np.mean(rgb)

In [ ]:
####### BORDER #########

def area_perimeter(mask): 
    '''Measures the area and perimeter of a mask image'''

    #area: the sum of all white pixels in the mask image
    area = np.sum(np.where(np.array(mask)==255, 1, 0))

    #perimeter: first find which pixels belong to the perimeter.
    struct_el = morphology.disk(1)
    mask_eroded = morphology.binary_erosion(mask, struct_el)
    image_perimeter = mask - mask_eroded

    #now we have the perimeter image, the sum of all white pixels in it
    perimeter = np.sum(image_perimeter)

    return area, perimeter

 
def get_compactness(mask):
    '''Computes and returns the compactness of a figure'''

    area, perimeter = area_perimeter(mask)

    return round(4*np.pi*area / perimeter ** 2, 4)


get_compactness(mask)

### Feature extraction

In [2]:
features = ['Asymmetry', 'Compactness', 'Color Variability']
features_functions = ['total_asymmetry', xxx , xxxx]

for image in images:
    
extractors = [get_symmetry]
feature_names = [symmetry, ...]
for image in images:
    features = []
    mask = load_mask()
    for feature_extractor in extractors:
        v = feature_extractor(mask, image)
        features.append(v)
    
    data.append(features)


# Save the features to csv
df = pd.DataFrame(features, columns = feature_names)
df.to_csv(os.path.join(data/transformed/features.csv))
"""

Hello from symmetry
